In [2]:
import os, getpass
from pandas import read_csv

credentials = {
    "url": "https://us-south.ml.cloud.ibm.com",
    "apikey": getpass.getpass("Please enter your WML api key (hit enter): ")
}




Please enter your WML api key (hit enter):  ········


In [3]:
try:
    project_id = os.environ["PROJECT_ID"]
except KeyError:
    project_id = input("Please enter your project_id (hit enter): ")

project_id



'0bf853a5-0690-4f8b-9af6-348ee7ea21bd'

In [4]:
import os, types
import pandas as pd
from ibm_boto3 import client
from botocore.client import Config
import ibm_boto3

def _iter_(self): return 0



In [ ]:
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='enter your API key here',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us-south.cloud-object-storage.appdomain.cloud'
)

bucket = 'handson-rdpib6dd78tge0x'
object_key = 'college_feedback_training.csv'

body = cos_client.get_object(Bucket=bucket, Key=object_key)['Body']

train_data = pd.read_csv(body)
train_data.head(5)


,Sl. No,Graduation Year,Gender,Feedback,Themes
0,1,2020,Prefer not to say,There are too many bureaucratic hurdles when t...,Administration
1,2,2023,Prefer not to say,"The Inaya Mittal is an excellent lecturer, alw...",Academics
2,3,2020,Male,The labs for distinctio were well-equipped and...,Academics
3,4,2023,Prefer not to say,The laboratory equipment for similique needs a...,Facilities
4,5,2021,Female,The process for obtaining transcripts is slow ...,Administration


In [ ]:
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='enter your API key here',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us-south.cloud-object-storage.appdomain.cloud'
)

bucket = 'handson-rdpib6dd78tge0x'
object_key = 'college_feedback_test.csv'

body = cos_client.get_object(Bucket=bucket, Key=object_key)['Body']

# add missing _iter_ method, so pandas accepts body as file-like object
if not hasattr(body, "_iter"): body.iter_ = types.MethodType(_iter_, body)

test_data = pd.read_csv(body)
test_data.head(5)



,Sl. No,Graduation Year,Gender,Feedback
0,1,2022,Female,I found the teaching methods for ratione to be...
1,2,2024,Male,There are too many bureaucratic hurdles when t...
2,3,2020,Male,I found the teaching methods for totam to be u...
3,4,2022,Prefer not to say,The student support services are very responsi...
4,5,2023,Non-binary,More transparency is needed in how student fee...


In [10]:
from ibm_watson_machine_learning.foundation_models.utils.enums import ModelTypes

model_id = ModelTypes.FLAN_UL2

satisfaction_instruction = """
Classify the given feedback into the given departments.\n Departments are Administrative, Academics and Facilities.\n

Feedback: found the teaching methods for odit to be uninspiring. More interactive sessions would be great.\n
Classification: Academics\n\n

Feedback: I appreciate the cleanliness of the washrooms, but more regular stocking of supplies is needed.\n
Classification: Facilities\n\n

Feedback: The admission process was smooth and the staff were very helpful.\n
Classification: Administrative\n\n
"""


In [11]:
from ibm_watson_machine_learning.metanames import GenTextParamsMetaNames as GenParams

parameters = {
    GenParams.MAX_NEW_TOKENS: 20
}

from ibm_watson_machine_learning.foundation_models import Model

model = Model(
    model_id=model_id,
    params=parameters,
    credentials=credentials,
    project_id=project_id
)



/opt/conda/envs/Python-RT24.1/lib/python3.11/site-packages/ibm_watson_machine_learning/foundation_models/utils/utils.py:273: LifecycleWarning: Model 'google/flan-ul2' is in deprecated state from 2025-05-28 until 2025-07-30. IDs of alternative models: None. Further details: https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/fm-model-lifecycle.html?context=wx&audience=wdp
  warnings.warn(default_warning_template.format(


In [13]:
import time

results = []
feedback = list(train_data.Feedback)
classification = list(train_data.Themes)

for input_text in feedback:
    prompt = " ".join([satisfaction_instruction, input_text])
    results.append(model.generate_text(prompt=prompt))
    time.sleep(0.6)  # Sleep to avoid hitting the rate limit
    feedback = list(train_data.Feedback)
    classification = list(train_data.Themes)    



In [14]:
feedback

['There are too many bureaucratic hurdles when trying to get approvals for student events.',
 'The Inaya Mittal is an excellent lecturer, always willing to help students.',
 'The labs for distinctio were well-equipped and provided great hands-on experience.',
 'The laboratory equipment for similique needs an upgrade; some machines are quite old.',
 'The process for obtaining transcripts is slow and cumbersome.',
 'The curriculum in voluptas was very engaging and well-structured. I learned a lot.',
 'The laboratory equipment for placeat needs an upgrade; some machines are quite old.',
 'The laboratory equipment for enim needs an upgrade; some machines are quite old.',
 'The campus greenery is beautiful and well-kept, providing a pleasant environment.',
 'The admission process was smooth and the staff were very helpful.',
 "There aren't enough charging points in common areas, which is inconvenient for laptop users.",
 'The campus greenery is beautiful and well-kept, providing a pleasant 

In [16]:
classification

['Administration',
 'Academics',
 'Academics',
 'Facilities',
 'Administration',
 'Academics',
 'Facilities',
 'Facilities',
 'Facilities',
 'Administration',
 'Facilities',
 'Facilities',
 'Academics',
 'Facilities',
 'Administration',
 'Academics',
 'Academics',
 'Facilities',
 'Administration',
 'Facilities',
 'Administration',
 'Facilities',
 'Facilities',
 'Facilities',
 'Administration',
 'Administration',
 'Facilities',
 'Administration',
 'Academics',
 'Facilities',
 'Facilities',
 'Administration',
 'Academics',
 'Facilities',
 'Academics',
 'Academics',
 'Facilities',
 'Facilities',
 'Facilities',
 'Administration',
 'Administration',
 'Administration',
 'Administration',
 'Administration',
 'Academics',
 'Facilities',
 'Academics',
 'Facilities',
 'Administration',
 'Administration',
 'Administration',
 'Administration',
 'Academics',
 'Facilities',
 'Administration',
 'Academics',
 'Facilities',
 'Facilities',
 'Facilities',
 'Academics',
 'Administration',
 'Facilities',
 

In [17]:
results

['Administrative',
 'Academics',
 'Academics',
 'Academics',
 'Administrative',
 'Academics',
 'Academics',
 'Academics',
 'Facilities',
 'Administrative',
 'Facilities',
 'Facilities',
 'Academics',
 'Facilities',
 'Academics',
 'Academics',
 'Administrative',
 'Facilities',
 'Administrative',
 'Facilities',
 'Administrative',
 'Facilities',
 'Facilities',
 'Facilities',
 'Administrative',
 'Administrative',
 'Facilities',
 'Administrative',
 'Academics',
 'Facilities',
 'Facilities',
 'Administrative',
 'Academics',
 'Facilities',
 'Academics',
 'Academics',
 'Facilities',
 'Facilities',
 'Facilities',
 'Administrative',
 'Academics',
 'Administrative',
 'Administrative',
 'Administrative',
 'Academics',
 'Facilities',
 'Administrative',
 'Facilities',
 'Administrative',
 'Administrative',
 'Administrative',
 'Administrative',
 'Academics',
 'Facilities',
 'Administrative',
 'Academics',
 'Facilities',
 'Academics',
 'Facilities',
 'Academics',
 'Administrative',
 'Facilities',
 'Fac

In [20]:
results = []
feedback_test = list(test_data.Feedback)

for input_text in feedback:
    prompt = " ".join([satisfaction_instruction, input_text])
    results.append(model.generate_text(prompt=prompt))
    time.sleep(0.6)  # Sleep to avoid hitting the rate limit
    feedback_test = list(train_data.Feedback)



In [21]:
feedback_test


['There are too many bureaucratic hurdles when trying to get approvals for student events.',
 'The Inaya Mittal is an excellent lecturer, always willing to help students.',
 'The labs for distinctio were well-equipped and provided great hands-on experience.',
 'The laboratory equipment for similique needs an upgrade; some machines are quite old.',
 'The process for obtaining transcripts is slow and cumbersome.',
 'The curriculum in voluptas was very engaging and well-structured. I learned a lot.',
 'The laboratory equipment for placeat needs an upgrade; some machines are quite old.',
 'The laboratory equipment for enim needs an upgrade; some machines are quite old.',
 'The campus greenery is beautiful and well-kept, providing a pleasant environment.',
 'The admission process was smooth and the staff were very helpful.',
 "There aren't enough charging points in common areas, which is inconvenient for laptop users.",
 'The campus greenery is beautiful and well-kept, providing a pleasant 

In [22]:
results

['Administrative',
 'Academics',
 'Academics',
 'Academics',
 'Administrative',
 'Academics',
 'Academics',
 'Academics',
 'Facilities',
 'Administrative',
 'Facilities',
 'Facilities',
 'Academics',
 'Facilities',
 'Academics',
 'Academics',
 'Administrative',
 'Facilities',
 'Administrative',
 'Facilities',
 'Administrative',
 'Facilities',
 'Facilities',
 'Facilities',
 'Administrative',
 'Administrative',
 'Facilities',
 'Administrative',
 'Academics',
 'Facilities',
 'Facilities',
 'Administrative',
 'Academics',
 'Facilities',
 'Academics',
 'Academics',
 'Facilities',
 'Facilities',
 'Facilities',
 'Administrative',
 'Academics',
 'Administrative',
 'Administrative',
 'Administrative',
 'Academics',
 'Facilities',
 'Administrative',
 'Facilities',
 'Administrative',
 'Administrative',
 'Administrative',
 'Administrative',
 'Academics',
 'Facilities',
 'Administrative',
 'Academics',
 'Facilities',
 'Academics',
 'Facilities',
 'Academics',
 'Administrative',
 'Facilities',
 'Fac